In [1]:
using CSV
using Distances
using Random

In [2]:
Random.seed!(3)
n = 20      # The number of patients
c_2 = 5     # unit distance cost
patients = [2:n+1;]
V = [1:n+1;]  # 1 denotes depot.    
q = [rand(12:15,n+1);]   # Requirment of each patient
arcs = []
for i in V
    for j in V 
        push!(arcs,(i,j))
    end
end
Q = 100    # Capacity
D = 10  # Max flying distance
q

21-element Array{Int64,1}:
 12
 15
 12
 13
 15
 15
 12
 15
 12
 14
 12
 15
 13
 12
 15
 15
 12
 12
 13
 15
 13

In [3]:
data = CSV.read("Patients21206_TableToExcel.csv")

,OID,CID,POINT_X,POINT_Y
,Int64,Int64,Float64,Float64
1,0,0,-76.5412,39.3452
2,1,1,-76.5244,39.3301
3,2,1,-76.5419,39.3376
4,3,1,-76.5142,39.3431
5,4,1,-76.5317,39.3358
6,5,1,-76.5589,39.3261
7,6,1,-76.5592,39.3222
8,7,1,-76.5357,39.3538
9,8,1,-76.5585,39.3379


In [4]:
x_loc = data[:,3];
y_loc = data[:,4];
d = zeros(n+1,n+1)   # Distance matrix
for i = 1: length(V)
    for j = 1: length(V)
        d[i,j] = haversine((x_loc[i],y_loc[i]),(x_loc[j],y_loc[j]),6372.8) 
    end
end
d

21×21 Array{Float64,2}:
 0.0       2.21155   0.84739   2.32869  …  0.695232  0.657811  1.40216
 2.21155   0.0       1.71839   1.68895     2.50207   2.38791   3.55578
 0.84739   1.71839   0.0       2.45652     0.803717  1.42599   2.20423
 2.32869   1.68895   2.45652   0.0         2.95289   2.01247   3.19105
 1.32232   0.893088  0.896458  1.70918     1.63575   1.57827   2.68982
 2.61453   3.00113   1.94337   4.28288  …  2.04497   3.26933   3.60205
 2.9887    3.11793   2.267     4.51098     2.45035   3.63729   4.02386
 1.06558   2.80901   1.87882   2.1995      1.65924   0.468407  1.00563
 1.69676   3.05396   1.42654   3.84949     1.0056    2.32737   2.3662 
 0.821159  2.83799   1.16323   3.14893     0.368652  1.33823   1.33095
 2.46379   2.54617   1.69898   3.91013  …  1.99211   3.09939   3.59885
 1.43642   2.42      0.889676  3.34535     0.873091  2.09381   2.4711 
 2.332     3.12128   2.96325   1.71407     3.01243   1.69015   2.32428
 2.26646   3.44673   3.00354   2.19558     2.90142   

In [5]:
using JuMP, Gurobi 
v = 1:5  # # of drones
m = Model(Gurobi.Optimizer);
@variable(m,x[arcs,v],Bin) #arcs covered by vehicle k
@variable(m,u[1:n+1,v])
# Each patient will be visited exactly once
@constraint(m,single1[j in patients],sum(x[(i,j),k] for i in V for k in v)==1) 
@constraint(m,single2[i in patients],sum(x[(i,j),k] for j in V for k in v)==1) 
# Drone starts from depot
@constraint(m,depot1[k in v],sum(x[(1,j),k] for j in patients)==1)
# Drone ends at depot
@constraint(m,depot2[k in v],sum(x[(i,1),k] for i in patients)==1)
# Capacity constriant
@constraint(m,capacity[k in v],sum(q[i]*x[(i,j),k] for i in patients for j in V)<=Q)
# Balance constraint
@constraint(m,balance[h in patients,k in v],sum(x[(i,h),k] for i in V)-sum(x[(h,j),k] for j in V)==0) 
# Subtour Elimination MTZ
@constraint(m,subtour1[k in v], u[1,k]==1)
@constraint(m,subtour2[k in v,i in 2:n+1], u[i,k]>=2)   
@constraint(m,subtour3[k in v,i in 2:n+1], u[i,k]<=n)
@constraint(m,subtour4[k in v,i in 2:n+1, j in 2:n+1], (u[i,k]-u[j,k]+1)<=(n-1)*(1-x[(i,j),k]))
# Maximum flying distance for each drone
@constraint(m,maxdis[k in v], sum(d[i,j]*x[(i,j),k] for i in V for j in V)<=D)                                             
# Objective Function
@objective(m,Min,5*sum(d[i,j]*x[(i,j),k] for i in V for j in V for k in v))                                                              
optimize!(m)
@show objective_value(m);

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 2365 rows, 2310 columns and 18605 nonzeros
Model fingerprint: 0x2c5c7d09
Variable types: 105 continuous, 2205 integer (2205 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 305 rows and 110 columns
Presolve time: 0.03s
Presolved: 2060 rows, 2200 columns, 17890 nonzeros
Variable types: 100 continuous, 2100 integer (2100 binary)

Root relaxation: objective 8.775078e+01, 111 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   87.75078    0   32          -   87.75078      -     -    0s
H    0     0                     191.2524806   87.75078  54.1%    

In [6]:
val = value.(x)
for k in v
    for i in V
        for j in V
            if val[(i,j),k]==1
                println(x[(i,j),k])
            end 
        end
    end
end

x[(1, 20),1]
x[(8, 21),1]
x[(20, 8),1]
x[(21, 1),1]
x[(1, 5),2]
x[(2, 4),2]
x[(4, 13),2]
x[(5, 15),2]
x[(13, 14),2]
x[(14, 17),2]
x[(15, 2),2]
x[(17, 1),2]
x[(1, 12),3]
x[(3, 1),3]
x[(6, 7),3]
x[(7, 11),3]
x[(9, 6),3]
x[(11, 3),3]
x[(12, 16),3]
x[(16, 9),3]
x[(1, 10),4]
x[(10, 19),4]
x[(19, 1),4]
x[(1, 18),5]
x[(18, 1),5]
